## Exercise 1: Scraping table from Wikipedia and transforming it to a dataframe

I first saved the Wikipedia page in HTML format onto my laptop and passed it to BeautifulSoup so that I can extract the table from the webpage. 

In [43]:
from bs4 import BeautifulSoup
import requests

with open('Canada post code.html') as html_file:
	soup = BeautifulSoup(html_file, 'lxml')

output = soup.find('div', class_='mw-parser-output')
table = output.table.tbody.text    # returns a string
#print(table)  


After extracting the table, the first thing that needs to be done is to convert this information into a Python list that contains the following three attributes: postal code, borough, and neighborhood. 

In [44]:
list_ = table.split('\n')
new_list = [i for i in list_ if i != '']
new_list

p =[]
b =[]
n =[]
for i in range(3,len(new_list),3):
    if new_list[i+1] == 'Not assigned':
        continue  
    
    p.append(new_list[i])
    b.append(new_list[i+1])
    if new_list[i+2] != 'Not assigned':
        n.append(new_list[i+2])
    else:
        n.append(new_list[i+1])  # set neighborhood the same name as borough 
#print(p[:10], '\n', b[:10], '\n', n[:10])
  
row = []
for i in range(len(p)):
    row.append([p[i], b[i], n[i]])
#row

The next step is to convert the list to a pandas dataframe. 

In [45]:
import pandas as pd

columns=['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(row, columns=columns)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


Obviously, this dataframe contains som duplicate values in the PostalCode column, so let's join the neighborhoods that have the same postal codes and then remove the duplicate rows.

In [46]:
#Check duplicate rows:
# df[df.duplicated(['PostalCode'])].sort_values(by='PostalCode').head(10)

In [47]:
# join neighborhoods that have the same postal code 
df_new = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(','.join).to_frame().reset_index()
df_new.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [48]:
df_new.shape

(103, 3)

**This is the solution to exercise 1.** 

##  Exercise 2: Adding latitude and the longitude information

In [49]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo.columns = ['PostalCode', 'Latitude', 'Longitude']
#df_geo.set_index('PostalCode', inplace=True)
df_geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [50]:
df_all = df_new.join(df_geo.set_index('PostalCode'), on='PostalCode')
df_all.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


**This is the solution to exercise 2.** 

## Exercise 3: Step 1 -- Visualizing the Neighborhoods

In the previous dataframe, I assigned the name Queen's Park, the same name as the borough, to one of the rows whose neighborhood information is missing. But this has resulted in two neighborhoods having the same name.I decided to assign a new name to this neighborhood to prevent confusions in future operations. 

In [51]:
df_all.loc[93, 'Neighborhood'] = 'Queen\'s Park Downtown Toronto'

In [52]:
duplicateRowsDF = df_all[df_all.duplicated(subset='Neighborhood')]
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

Duplicate Rows except first occurrence based on all columns are :
Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood, Latitude, Longitude]
Index: []


Now we don't any any duplicate names and can move on to visualizing the neighborhoods on a map. 

In [53]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [54]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="TRT_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [55]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_all['Latitude'], df_all['Longitude'], df_all['Borough'], df_all['Neighborhood']):
  
    label = borough.upper() + ' : ' + neighborhood
    #'{}, \n {}'.format(borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Step 2 -- Define Foursquare Credentials and Version

In [56]:
CLIENT_ID = 'DPNNZEDGIAFE3RQAUSL0BV1YTLQFG3VGUFKD45HQAU3M0BJX' # your Foursquare ID
CLIENT_SECRET = 'BU5ZOGKYNLV1ESWZGXMTU2DWC5QCM4DOZN3CTCPOXDCVECLY' # your Foursquare Secret
VERSION = '20191210' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DPNNZEDGIAFE3RQAUSL0BV1YTLQFG3VGUFKD45HQAU3M0BJX
CLIENT_SECRET:BU5ZOGKYNLV1ESWZGXMTU2DWC5QCM4DOZN3CTCPOXDCVECLY


In [57]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [58]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    
    # return a list first, then transform the entire list to a dataframe. 
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    
    return(nearby_venues)

In [82]:
Toronto_venues = getNearbyVenues(names=df_all['Neighborhood'],
                                   latitudes=df_all['Latitude'],
                                   longitudes=df_all['Longitude']
                                  )

In [83]:
Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge,Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target


### Step 3 -- Analyze Each Neighborhood

In [84]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
#print(Toronto_onehot)
# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 
Toronto_onehot.set_index('Neighborhood', inplace=True)
Toronto_onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

In [85]:
#added code 
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').sum().reset_index()


Let's analyze the first neighborhood and print the top 5 venues in that area.

In [86]:
#####
temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == 'Adelaide,King,Richmond'].T.reset_index()
temp.columns = ['venue','counts']
temp = temp.iloc[1:]
temp['counts'] = temp['counts'].astype(float)
temp = temp.round({'counts': 2})
temp.sort_values('counts', ascending=False).head().reset_index(drop=True)

,venue,counts
0,Coffee Shop,7.0
1,Café,5.0
2,Steakhouse,4.0
3,Thai Restaurant,4.0
4,Restaurant,3.0


I changed the following cell to Markdown to avoid the output, which is too long and not good for displaying the code. 

num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','counts']
    temp = temp.iloc[1:]
    temp['counts'] = temp['counts'].astype(float)
    temp = temp.round({'counts': 2})
    print(temp.sort_values('counts', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [105]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [106]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Restaurant,Bakery,Sushi Restaurant,Bar,Salad Place,Asian Restaurant
1,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Clothing Store,Breakfast Spot,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Arts & Crafts Store,Playground,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Fried Chicken Joint,Pharmacy,Video Store,Beer Store,Pizza Place,Fast Food Restaurant,Sandwich Place,Discount Store,Department Store
4,"Alderwood,Long Branch",Pizza Place,Skating Rink,Pharmacy,Pub,Sandwich Place,Dance Studio,Gym,Coffee Shop,Donut Shop,Drugstore


### Step 4 -- Cluster Neighborhoods

In [90]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_#[0:10] 

array([1, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 1, 3, 2,
       0, 2, 2, 2, 2, 1, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4,
       1, 2, 2, 2, 2, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0,
       2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 4, 2, 2, 1, 1, 0, 0, 2, 2,
       0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2], dtype=int32)

In [108]:
#Toronto_data
#neighborhoods_venues_sorted.drop(columns='Cluster Labels', inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Restaurant,Bakery,Sushi Restaurant,Bar,Salad Place,Asian Restaurant
1,2,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Clothing Store,Breakfast Spot,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
2,2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Arts & Crafts Store,Playground,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
3,2,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Fried Chicken Joint,Pharmacy,Video Store,Beer Store,Pizza Place,Fast Food Restaurant,Sandwich Place,Discount Store,Department Store
4,2,"Alderwood,Long Branch",Pizza Place,Skating Rink,Pharmacy,Pub,Sandwich Place,Dance Studio,Gym,Coffee Shop,Donut Shop,Drugstore


The next step is to merge the **neighborhoods_venues_sorted** dataframe with the **df_all** dataframe. But first we need to check if their dimensions are consistent

In [109]:
print(df_all.shape)
print(neighborhoods_venues_sorted.shape)

(103, 5)
(101, 12)


We can see that the former dataframe has two more rows, i.e. two more neighborhoods, than the latter dataframe. This occurs because no recommended venues were found near these two neighborhoods within the specified distcance, and will cause issues when merging with another dataframe. Therefore, we need to find out what these two neighborhoods are and remove them from our dataframe. 

In [110]:
list1= df_all['Neighborhood'].values.tolist()
list2= neighborhoods_venues_sorted['Neighborhood'].values.tolist()

for i in list1:
    if i not in list2:
        print(i)

Upper Rouge
Queen's Park Downtown Toronto


In [111]:
df_all[df_all['Neighborhood'] == 'Upper Rouge']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
16,M1X,Scarborough,Upper Rouge,43.836125,-79.205636


In [112]:
df_all[df_all['Neighborhood'] == 'Queen\'s Park Downtown Toronto']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
93,M9A,Downtown Toronto,Queen's Park Downtown Toronto,43.667856,-79.532242


In [113]:
# let's drop these two neighborhoods
df_all_dropped = df_all.drop([16,93], axis=0).reset_index()

In [114]:
Toronto_merged = df_all_dropped

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Print Shop,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
1,1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,2,Construction & Landscaping,Moving Target,Bar,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
2,2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,2,Electronics Store,Medical Center,Mexican Restaurant,Spa,Breakfast Spot,Pizza Place,Rental Car Location,Intersection,Department Store,Dessert Shop
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Pharmacy,Korean Restaurant,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,Fried Chicken Joint,Thai Restaurant,Bank,Athletics & Sports,Caribbean Restaurant,Gas Station,Hakka Restaurant,Bakery,Dumpling Restaurant,Drugstore


In [115]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
print('x:', x)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
print('ys:', ys)
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#print(colors_array)
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)

# add markers to the map
markers_colors = []
for lat, lon, nb, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(nb) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

x: [0 1 2 3 4]
ys: [array([0, 1, 2, 3, 4]), array([ 1,  3,  7, 13, 21]), array([ 2,  7, 20, 41, 70]), array([  3,  13,  41,  87, 151]), array([  4,  21,  70, 151, 264])]
['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


In [116]:
neighborhoods_venues_sorted[neighborhoods_venues_sorted['Neighborhood'] == 'Fairview,Henry Farm,Oriole']

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,4,"Fairview,Henry Farm,Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Tea Room,Food Court,Bakery,Japanese Restaurant,Bus Station,Burger Joint


### Step 5 -- Examine the clusters

#### Cluster 1

In [117]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, 
                     Toronto_merged.columns[[3] + list(range(7, Toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Willowdale South,Coffee Shop,Ramen Restaurant,Café,Pizza Place,Japanese Restaurant,Restaurant,Sushi Restaurant,Sandwich Place,Fast Food Restaurant,Steakhouse
37,Leaside,Coffee Shop,Sporting Goods Shop,Furniture / Home Store,Burger Joint,Grocery Store,Breakfast Spot,Fish & Chips Shop,Sports Bar,Beer Store,Smoothie Shop
40,"The Danforth West,Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Bookstore,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place
42,Studio District,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Brewery,Sandwich Place
46,Davisville,Dessert Shop,Sandwich Place,Pizza Place,Sushi Restaurant,Italian Restaurant,Coffee Shop,Gym,Café,Fried Chicken Joint,Seafood Restaurant
50,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Flower Shop,Italian Restaurant,Bakery,Pub,Café,Pizza Place,Grocery Store,Pet Store
51,Church and Wellesley,Sushi Restaurant,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Café,Hotel,Mediterranean Restaurant,Gym,Gastropub
52,Harbourfront,Coffee Shop,Park,Bakery,Pub,Restaurant,Breakfast Spot,Café,Mexican Restaurant,Theater,Hotel
55,Berczy Park,Coffee Shop,Steakhouse,Cocktail Bar,Café,Beer Bar,Cheese Shop,Seafood Restaurant,Bakery,Farmers Market,Hotel
61,"Bedford Park,Lawrence Manor East",Sushi Restaurant,Coffee Shop,Italian Restaurant,Grocery Store,Butcher,Café,Restaurant,Hobby Shop,Indian Restaurant,Fast Food Restaurant


#### Clusters 2 - 5

In [118]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, 
                     Toronto_merged.columns[[3] + list(range(7, Toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,St. James Town,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Cosmetics Shop,Beer Bar,Breakfast Spot,Diner,Italian Restaurant
56,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Ice Cream Shop,Burger Joint,Café,Chinese Restaurant,Juice Bar,Japanese Restaurant,Fried Chicken Joint
57,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Restaurant,Bakery,Sushi Restaurant,Bar,Salad Place,Asian Restaurant
58,"Harbourfront East,Toronto Islands,Union Station",Coffee Shop,Aquarium,Café,Hotel,Italian Restaurant,Scenic Lookout,Fried Chicken Joint,Restaurant,Brewery,Bakery
59,"Design Exchange,Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,Bar,Bakery,Seafood Restaurant,Gastropub,Deli / Bodega,American Restaurant
60,"Commerce Court,Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Deli / Bodega,Italian Restaurant,Gym,Bakery,Gastropub
68,Stn A PO Boxes 25 The Esplanade,Coffee Shop,Café,Restaurant,Hotel,Beer Bar,Italian Restaurant,Japanese Restaurant,Seafood Restaurant,Cocktail Bar,Bakery
69,"First Canadian Place,Underground city",Coffee Shop,Café,Restaurant,Steakhouse,Hotel,Seafood Restaurant,Japanese Restaurant,Bar,Asian Restaurant,Deli / Bodega


In [119]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, 
                     Toronto_merged.columns[[3] + list(range(7, Toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge,Malvern",Fast Food Restaurant,Print Shop,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
1,"Highland Creek,Rouge Hill,Port Union",Construction & Landscaping,Moving Target,Bar,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
2,"Guildwood,Morningside,West Hill",Electronics Store,Medical Center,Mexican Restaurant,Spa,Breakfast Spot,Pizza Place,Rental Car Location,Intersection,Department Store,Dessert Shop
3,Woburn,Coffee Shop,Pharmacy,Korean Restaurant,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
4,Cedarbrae,Fried Chicken Joint,Thai Restaurant,Bank,Athletics & Sports,Caribbean Restaurant,Gas Station,Hakka Restaurant,Bakery,Dumpling Restaurant,Drugstore
5,Scarborough Village,Playground,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
6,"East Birchmount Park,Ionview,Kennedy Park",Discount Store,Chinese Restaurant,Convenience Store,Bus Station,Hobby Shop,Department Store,Coffee Shop,Eastern European Restaurant,Electronics Store,Dumpling Restaurant
7,"Clairlea,Golden Mile,Oakridge",Bakery,Bus Line,Soccer Field,Metro Station,Bus Station,Fast Food Restaurant,Intersection,Park,Gas Station,Dance Studio
8,"Cliffcrest,Cliffside,Scarborough Village West",American Restaurant,Motel,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio
9,"Birch Cliff,Cliffside West",Café,College Stadium,Skating Rink,General Entertainment,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


In [120]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, 
                     Toronto_merged.columns[[3] + list(range(7, Toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,"Chinatown,Grange Park,Kensington Market",Café,Bar,Bakery,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Chinese Restaurant,Dumpling Restaurant,Vegetarian / Vegan Restaurant,Grocery Store


In [121]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, 
                     Toronto_merged.columns[[3] + list(range(7, Toronto_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,"Fairview,Henry Farm,Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Tea Room,Food Court,Bakery,Japanese Restaurant,Bus Station,Burger Joint
53,"Ryerson,Garden District",Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Fast Food Restaurant,Café,Restaurant,Bubble Tea Shop,Pizza Place,Bookstore
